In [ ]:
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pathlib
import os

MODEL = pathlib.Path('wall_recognizer.keras')
LABELLED = pathlib.Path('Dataset', 'Wall Recognizer', 'Labelled')

In [ ]:
BUFFER = 128

file_paths = tf.data.Dataset.list_files(str(LABELLED/'*.png'), shuffle=False).shuffle(BUFFER, reshuffle_each_iteration=False)

In [ ]:
def parse_images(file_path):
    # extract label
    label = tf.strings.split(file_path, os.path.sep)[-1]
    label = tf.strings.split(label, '_')[:-1]
    label = tf.strings.to_number(label, tf.int32)

    # load image
    image = tf.io.read_file(file_path)
    image = tf.io.decode_png(image, channels=1)
    image = tf.image.resize(image, [28, 28])
    image = tf.cast(image, tf.float32) / 255
    return image, label

dataset = file_paths.map(parse_images)

In [ ]:
train_size = 200

train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

In [ ]:
image = iter(train_dataset).__next__()
print(image[0].shape)

In [ ]:
plt.figure(figsize=(15, 5))
for i, (image, label) in zip(range(3), iter(train_dataset)):
    label = label.numpy().astype(np.uint8).tolist()

    plt.subplot(1, 3, i + 1)
    plt.imshow(image, cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title('_'.join([str(x) for x in label]))
plt.show()

In [ ]:
train_dataset = train_dataset.batch(256)
test_dataset = test_dataset.batch(256)

In [ ]:
def get_model():
    model = keras.Sequential()

    # input layer
    model.add(keras.layers.InputLayer(input_shape=(28, 28, 1)))

    # first convolution layer
    model.add(keras.layers.Conv2D(32, (5, 5), padding='same', kernel_initializer='he_uniform'))
    model.add(keras.layers.ReLU())
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

    # second convolution layer
    model.add(keras.layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform'))
    model.add(keras.layers.ReLU())
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

    # transition to fully connected layers
    model.add(keras.layers.Flatten())

    # first dense layer
    model.add(keras.layers.Dense(64, kernel_initializer='he_uniform'))
    model.add(keras.layers.ReLU())
    model.add(keras.layers.Dropout(0.5))

    # second dense layer
    model.add(keras.layers.Dense(64, kernel_initializer='he_uniform'))
    model.add(keras.layers.ReLU())
    model.add(keras.layers.Dropout(0.5))

    # output layer
    model.add(keras.layers.Dense(4, kernel_initializer='he_uniform', activation='sigmoid'))
    return model

In [ ]:
# create a model
model = get_model()
print(model.summary())
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=keras.losses.BinaryCrossentropy(from_logits=False),
    metrics='accuracy'
)

In [ ]:
for file in os.listdir('Checkpoints'):
    os.remove(f'Checkpoints/{file}')

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='Checkpoints/{epoch:02d}-{val_loss:.2f}.hdf5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

In [ ]:
history = model.fit(train_dataset, epochs=100, validation_data=test_dataset, callbacks=[model_checkpoint_callback])

In [ ]:
model.evaluate(test_dataset)

In [ ]:
predictions = model.predict(test_dataset) < 0.5
predictions